**Reference**:

https://towardsdatascience.com/how-to-fine-tune-an-nlp-regression-model-with-transformers-and-huggingface-94b2ed6f798f

https://predictivehacks.com/how-to-fine-tune-an-nlp-regression-model-with-transformers-and-huggingface/


In [31]:
from transformers import AutoTokenizer

model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)



In [32]:
encoding = tokenizer("We are very happy to show you the 🤗 Transformers library.")
print(encoding)

{'input_ids': [101, 11312, 10320, 12495, 19308, 10114, 11391, 10855, 10103, 100, 58263, 13299, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [33]:
tokenizer(
    ["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."],
    padding=True,
    truncation=True,
    max_length=512,
)

{'input_ids': [[101, 11312, 10320, 12495, 19308, 10114, 11391, 10855, 10103, 100, 58263, 13299, 119, 102], [101, 11312, 18763, 10855, 11530, 112, 162, 39487, 10197, 119, 102, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]]}

In [34]:
import numpy as np
import pandas as pd
import transformers
from datasets import Dataset,load_dataset, load_from_disk
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [35]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

tokens = tokenizer('🚨 JUNE DROP LIVE 🚨')['input_ids']
actual_tokens = [tokenizer.decode(i) for i in tokenizer('🚨 JUNE DROP LIVE 🚨')['input_ids']]

print(f'tokens      :{tokens}')
print(f'actual token:{actual_tokens}')

for i in ['🚨', '🙂', '😍', '✌️' , '🤩 ']:
    tokenizer.add_tokens(i)
revised_actual_tokens = [tokenizer.decode(i) for i in tokenizer('🚨 JUNE DROP LIVE 🚨')['input_ids']]

# Now, if you tokenize the sentence you will see that the emoji remains as emoji and not the [UNK] token.
print(f'revised actual tokens:{revised_actual_tokens}')


tokens      :[101, 100, 2238, 4530, 2444, 100, 102]
actual token:['[CLS]', '[UNK]', 'june', 'drop', 'live', '[UNK]', '[SEP]']
revised actual tokens:['[CLS]', '🚨', 'june', 'drop', 'live', '🚨', '[SEP]']


In [36]:
# df_ = pd.read_csv('df_nlp_real.csv')
# df_.head(2)

In [37]:
import pandas as pd 

columns = ['text','wip','activity_nlp','resource_nlp','case_nlp']
def make_dataset(dataset, iloc_from, iloc_to):
    df_ = dataset[columns].iloc[iloc_from:iloc_to]  
    df_.rename(columns={"wip": "labels"})
    df_.reset_index(inplace=True,drop=True)
    return df_

df = pd.read_csv('df_nlp_real.csv',usecols=columns,dtype={'wip':'float'})
df_nlp = make_dataset(df, 200,300)
display(df_nlp.head())

,text,wip,activity_nlp,resource_nlp,case_nlp
0,R2 starts A3 on C1030 at 2010-02-22 12:05,48.0,A3,R2,C1030
1,R2 starts A1 on C1328 at 2010-02-22 13:00,0.0,A1,R2,C1328
2,R2 starts A2 on C1328 at 2010-02-22 13:01,49.0,A2,R2,C1328
3,R2 starts A1 on C3185 at 2010-02-23 08:23,0.0,A1,R2,C3185
4,R2 starts A2 on C3185 at 2010-02-23 08:24,50.0,A2,R2,C3185


In [38]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
token_columns = ['activity_nlp','resource_nlp','case_nlp']
for x in token_columns:
    for y in df_nlp[x].unique():
        tokenizer.add_tokens(y)
        
# df_nlp["token"] = df_nlp["text"].apply(lambda x: tokenizer(x , padding="max_length", truncation=True))
df_nlp["token"] = df_nlp["text"].apply(lambda x: tokenizer(x , padding=True, truncation=True))

ls = [df_nlp["token"][x]['input_ids'] for x in range(len(df_nlp["token"]))]
print(f'max len of tokens:{max([len(x) for x in ls])}')

text = df_nlp["text"][0]
tokens = tokenizer(text)['input_ids']
actual_tokens = [tokenizer.decode(i) for i in tokens]

print(f'text  :{text} \ntokens:{tokens} \nactual token:{actual_tokens}')

max len of tokens:16
text  :R2 starts A3 on C1030 at 2010-02-22 12:05 
tokens:[101, 30529, 4627, 30522, 2006, 30536, 2012, 2230, 1011, 6185, 1011, 2570, 2260, 1024, 5709, 102] 
actual token:['[CLS]', 'R2', 'starts', 'A3', 'on', 'C1030', 'at', '2010', '-', '02', '-', '22', '12', ':', '05', '[SEP]']


# Linear Regression 

In [39]:
import numpy as np
import pandas as pd

import transformers
from datasets import Dataset,load_dataset, load_from_disk
from transformers import AutoTokenizer, AutoModelForSequenceClassification


## Pandas To Dataset

In [48]:
import pandas as pd 

columns = ['text','wip','activity_nlp','resource_nlp','case_nlp']

def make_dataset(dataset, iloc_from, iloc_to):
    df_ = dataset[columns].iloc[iloc_from:iloc_to]  
    df_= df_.rename(columns={"wip": "labels"})
    df_.reset_index(inplace=True,drop=True)
    return df_

df = pd.read_csv('df_nlp_real.csv',usecols=columns,dtype={'wip':'float'})
df_nlp = make_dataset(df, 200,300)
display(df_nlp.head(3))

dataset = Dataset.from_pandas(df_nlp[['text','labels']],preserve_index=False) 
dataset = dataset.train_test_split(test_size=0.3) 



def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

dataset

,text,labels,activity_nlp,resource_nlp,case_nlp
0,R2 starts A3 on C1030 at 2010-02-22 12:05,48.0,A3,R2,C1030
1,R2 starts A1 on C1328 at 2010-02-22 13:00,0.0,A1,R2,C1328
2,R2 starts A2 on C1328 at 2010-02-22 13:01,49.0,A2,R2,C1328


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 70
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 30
    })
})

## Tokenization & How To Add New Tokens


In [53]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

token_columns = ['activity_nlp','resource_nlp','case_nlp']
for x in token_columns:
    for y in df_nlp[x].unique():
        tokenizer.add_tokens(y)
        
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

#### Test Tokenization

In [52]:
text = df_nlp["text"][0]
tokens = tokenizer(text)['input_ids']
actual_tokens = [tokenizer.decode(i) for i in tokens]

print(f'text  :{text} \ntokens:{tokens} \nactual token:{actual_tokens}')

text  :R2 starts A3 on C1030 at 2010-02-22 12:05 
tokens:[101, 30529, 4627, 30522, 2006, 30536, 2012, 2230, 1011, 6185, 1011, 2570, 2260, 1024, 5709, 102] 
actual token:['[CLS]', 'R2', 'starts', 'A3', 'on', 'C1030', 'at', '2010', '-', '02', '-', '22', '12', ':', '05', '[SEP]']


## Fine-Tuning The Model

In [44]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=1)
model.resize_token_embeddings(len(tokenizer))

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'pre_classifi

Embedding(30581, 768)

### Metrics Function


In [62]:
from datasets import load_metric
from sklearn.metrics import mean_squared_error

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    rmse = mean_squared_error(labels, predictions, squared=False)
    return {"rmse": rmse}

### Train The Model

In [61]:
from transformers import TrainingArguments, Trainer
import os 
os.environ["WANDB_DISABLED"] = "true"


training_args = TrainingArguments(output_dir="test_trainer",
                                  logging_strategy="epoch",
                                  evaluation_strategy="epoch",
                                  per_device_train_batch_size=16,
                                  per_device_eval_batch_size=16,
                                  num_train_epochs=3,
                                  save_total_limit = 2,
                                  save_strategy = 'no',
                                  load_best_model_at_end=False
                                  )


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics
)
trainer.train()


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/Users/yousef/miniforge3/envs/mlp/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim

Epoch,Training Loss,Validation Loss,Rmse
1,1840.289300,2063.958252,45.430809
2,1857.195500,2029.629028,45.051407
3,1835.005900,2017.985474,44.921997


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 30
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 30
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 30
 

TrainOutput(global_step=15, training_loss=1844.1635416666666, metrics={'train_runtime': 106.437, 'train_samples_per_second': 1.973, 'train_steps_per_second': 0.141, 'total_flos': 27817657620480.0, 'train_loss': 1844.1635416666666, 'epoch': 3.0})

## Save And Load The Pre-Trained Model And Tokenizer


In [ ]:
# save the model/tokenizer

model.save_pretrained("model")
tokenizer.save_pretrained("tokenizer")

# load the model/tokenizer

from transformers import AutoModelForTokenClassification
model = AutoModelForSequenceClassification.from_pretrained("model")
tokenizer = AutoTokenizer.from_pretrained("tokenizer")

##  Use The Model

In [58]:
from transformers import Trainer
trainer = Trainer(model=model)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True) 

def pipeline_prediction(text):
    df=pd.DataFrame({'text':[text]})
    dataset = Dataset.from_pandas(df,preserve_index=False) 
    tokenized_datasets = dataset.map(tokenize_function)
    raw_pred, _, _ = trainer.predict(tokenized_datasets) 
    return(raw_pred[0][0])

pipeline_prediction("🚨 Get 50% now!")

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


  0%|          | 0/1 [00:00<?, ?ex/s]

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1
  Batch size = 8


0.7360775